In [1]:
from binance.client import Client
import pandas as pd


api_key="mgMDvJNtetK1AukW4EOroL1Vq9dUIrkltYzBTATzXzCS9mX7a95nWIvvxoMAYcug"
api_secret="sBZlBhLd5G0wEVDX3v47lZuAAAsreY9PJDvPEZ1G7f11uCXPjb2likvAJwcbdeGb"

API_KEY = api_key
API_SECRET = api_secret

client = Client(api_key, api_secret)  # mainnet
symbol = "BTCUSDT"
interval = Client.KLINE_INTERVAL_1MINUTE
start_str = "1 Nov, 2025"

klines = client.get_historical_klines(symbol, interval, start_str)

df = pd.DataFrame(klines, columns=[
    "open_time","open","high","low","close","volume",
    "close_time","quote_asset_volume","number_of_trades",
    "taker_buy_base_asset_volume","taker_buy_quote_asset_volume","ignore"
])

df["open_time"] = pd.to_datetime(df["open_time"], unit='ms')
df["close_time"] = pd.to_datetime(df["close_time"], unit='ms')
for col in ["open","high","low","close","volume"]:
    df[col] = df[col].astype(float)

print(df.head())


            open_time       open       high        low      close    volume  \
0 2025-11-01 00:00:00  109608.01  109621.62  109608.00  109616.35   5.65926   
1 2025-11-01 00:01:00  109616.34  109639.64  109550.46  109550.47  47.19536   
2 2025-11-01 00:02:00  109550.46  109550.46  109470.92  109470.93  10.29554   
3 2025-11-01 00:03:00  109470.92  109476.12  109447.00  109447.00  39.15662   
4 2025-11-01 00:04:00  109447.00  109457.94  109441.26  109444.44   4.47815   

               close_time quote_asset_volume  number_of_trades  \
0 2025-11-01 00:00:59.999    620327.34965220              1005   
1 2025-11-01 00:01:59.999   5172298.30687730              2016   
2 2025-11-01 00:02:59.999   1127423.03182360              2208   
3 2025-11-01 00:03:59.999   4286187.85469350              2785   
4 2025-11-01 00:04:59.999    490126.77018130              1443   

  taker_buy_base_asset_volume taker_buy_quote_asset_volume ignore  
0                  3.81740000              418426.01560140  

In [3]:
df.shape()

TypeError: 'tuple' object is not callable

In [2]:
import numpy as np
import pandas as pd
import vectorbt as vbt
from itertools import product

def alpha_momentum_vbt(close, window, up_thresh, down_thresh):
    signals = np.zeros_like(close)
    for i in range(window, len(close)):
        mean_window = close[i-window:i].mean()
        if close[i] > mean_window * up_thresh:
            signals[i] = 1
        elif close[i] < mean_window * down_thresh:
            signals[i] = -1
    return signals

# Parameters
train_size = 1000
test_size = 200
param_grid = {
    'window': [5, 10, 15, 20],
    'up_thresh': [1.0005, 1.001, 1.002, 1.005],
    'down_thresh': [0.995, 0.997, 0.999, 0.9995]
}

df = df.set_index('open_time').asfreq('1min')
close = df['close'].values

results = []
n = len(close)
start = 0

while start + train_size + test_size < n:
    train = close[start:start+train_size]
    test = close[start+train_size:start+train_size+test_size]

    # --- Step 1: Optimize on training data ---
    best_sharpe = -np.inf
    best_params = None
    for w, up, down in product(param_grid['window'], param_grid['up_thresh'], param_grid['down_thresh']):
        signals = alpha_momentum_vbt(train, w, up, down)
        pf = vbt.Portfolio.from_signals(
            train,
            entries=signals==1,
            exits=signals==-1,
            init_cash=10000,
            fees=0.001,
            freq='1min'
        )
        sharpe = pf.sharpe_ratio()
        if sharpe > best_sharpe:
            best_sharpe = sharpe
            best_params = (w, up, down)

    # --- Step 2: Test on unseen data ---
    w, up, down = best_params
    test_signals = alpha_momentum_vbt(test, w, up, down)
    test_pf = vbt.Portfolio.from_signals(
        test,
        entries=test_signals==1,
        exits=test_signals==-1,
        init_cash=10000,
        fees=0.001,
        freq='1min'
    )

    results.append({
        'start': start,
        'best_window': w,
        'best_up_thresh': up,
        'best_down_thresh': down,
        'train_sharpe': best_sharpe,
        'test_return': test_pf.total_return(),
        'test_sharpe': test_pf.sharpe_ratio()
    })

    # Slide forward
    start += test_size

results_df = pd.DataFrame(results)
print(results_df)
print("\nAverage Out-of-Sample Sharpe:", results_df['test_sharpe'].mean())


edited manually, stop running 
    start  best_window  best_up_thresh  best_down_thresh  train_sharpe  \
0       0            5           1.005            0.9950           inf   
1     200            5           1.005            0.9950           inf   
2     400            5           1.005            0.9950           inf   
3     600            5           1.005            0.9950           inf   
4     800            5           1.005            0.9950           inf   
5    1000            5           1.002            0.9950           inf   
6    1200            5           1.002            0.9950           inf   
7    1400            5           1.002            0.9950           inf   
8    1600            5           1.002            0.9950           inf   
9    1800            5           1.002            0.9950           inf   
10   2000            5           1.005            0.9950           inf   
11   2200            5           1.005            0.9950           inf   
12   24